#Виконано за сприяння Рожанчука Богдана
#https://github.com/BogdanJeN/Geo/tree/main/Lab_07

In [1]:
import numpy as np
import rasterio as rastr
import matplotlib.pyplot as plt
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst

In [2]:
import json

with open('../field_polygons.geojson', 'r') as f:
    data = json.load(f)

for feature in data['features']:
    index = feature['properties']['id'] - 1
    name = feature['properties']['Name']
    feature['properties']['Name'] = f'{name}_{index}'

with open('fields_updated.geojson', 'w') as f:
    json.dump(data, f)

In [3]:
import geopandas as gpd

gdf = gpd.read_file('../Lab_07/fields_updated.geojson')

print(gdf[['Name']])

            Name
0         Pin4_0
1        Pin17_1
2    Shev7-8-9_2
3       Shev10_3
4          Tr5_4
5        Shev6_5
6          Tr4_6
7          Tr3_7
8          Tr2_8
9         Tr10_9
10        Tr9_10
11      Shev4_11
12        Tr1_12
13        Tr2_13
14     Yatz50_14
15  Yatz220/1_15
16  Yatz220/2_16
17     Yatz36_17
18     Yatz60_17
19     Yatz45_18
20     Yatz44_20
21        Ks8_21
22       Pin9_22
23        Ks1_23
24        Ks5_24
25     Pin9/1_26
26      pin10_26
27      Shev3_28
28      Shev2_29
29      Shev1_30
30       Ks11_31
31        Ks6_31
32        Ks7_33
33        Tr6_35
34       Tr10_35
35        Tr8_37
36        Tr8_37
37        Tr7_38
38      Tr7/1_39
39        Ls3_40
40        Ks2_41
41       Ks15_41
42       Ks12_81
43       Ks13_82


In [4]:
import glob

folder_path = '../spatial_data/' 
pattern = '**/*.tif'

files = glob.glob(f'{folder_path}/{pattern}', recursive=True)

print(f'Number of files with .tif extension: {len(files)}')

Number of files with .tif extension: 68


In [5]:
import os
import geopandas as gpd

fields = gpd.read_file("../Lab_07/fields_updated.geojson")

name_column = fields["Name"]

os.makedirs("resampled", exist_ok=True)

for name in name_column:
    entry_dir = os.path.join("resampled", name)
    os.makedirs(entry_dir, exist_ok=True)

In [6]:
import os
import geopandas as gpd
import rasterio as rastr
from rasterio.features import geometry_mask
from rasterio import mask as msk

fields = gpd.read_file("../Lab_07/fields_updated.geojson")

spatial_data = rastr.open("../spatial_data/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif")


for index, row in fields.iterrows():
    name = row["Name"] 
    geom = row["geometry"] 
    
    entry_dir = os.path.join("resampled", name)
    
    data, transform = rastr.mask.mask(spatial_data, [geom], crop=True)
    
    meta = spatial_data.meta.copy()
    
    meta.update({"driver": "GTiff",
                 "height": data.shape[1],
                 "width": data.shape[2],
                 "transform": transform})

    os.makedirs(entry_dir, exist_ok=True)
    
    with rastr.open(os.path.join(entry_dir, "example.tif"), "w", **meta) as dst:
        dst.write(data)

Результат Виконання.

```{"type": "FeatureCollection", "name": "fields_wsg84", "crs": {"type": "name", "properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}}, "features": [{"type": "Feature", "properties": {"id": 1, "Name": "Pin4_0"}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[30.174112718865484, 49.99640520439303], [30.177051214311422, 50.001295984584175], [30.193489148551734, 49.99510707973654], [30.192885666074687, 49.99335231760143], [30.190434598783277, 49.98909048582229], [30.186915831724775, 49.99101850412295], [30.185968828453095, 49.99021865451887], [30.17709299386753, 49.99532194416891], [30.174112718865484, 49.99640520439303]]]]}}```
